In [1]:
import pandas as pd

book_ratings = pd.read_csv('C:/Akshata/Courses/summer19/large_scale_analytics/Individual_project/BX-CSV-Dump/BX-Book-Ratings.csv',
                        error_bad_lines=False, engine="python", index_col=False, delimiter=";", quoting = 3)
book_ratings.columns = ['userid', 'ISBN', 'BookRatings']

books_info = pd.read_csv('C:/Akshata/Courses/summer19/large_scale_analytics/Individual_project/BX-CSV-Dump/BX-Books.csv',
                        error_bad_lines=False, engine="python",index_col=False, delimiter=";", quoting = 3)
books_info.drop(books_info.columns[[5,6,7]], axis = 1, inplace=True)
books_info.columns = ['ISBN', 'BookTitle', 'BookAuthor', 'YearOfPublication', 'Publisher']


In [2]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

books_info.ISBN = le.fit_transform(books_info.ISBN)
books_info.BookAuthor = le.fit_transform(books_info.BookAuthor)
books_info.YearOfPublication = le.fit_transform(books_info.YearOfPublication)
books_info.Publisher = le.fit_transform(books_info.Publisher)


book_ratings.userid = le.fit_transform(book_ratings.userid)
book_ratings.ISBN = le.fit_transform(book_ratings.ISBN)
book_ratings.BookRatings = le.fit_transform(book_ratings.BookRatings)

In [3]:
book_ratings.head()

,userid,ISBN,BookRatings
0,73842,57188,0
1,73843,29750,6
2,73844,107392,0
3,73845,127253,4
4,73845,127287,7


In [4]:
books_info.head()

,ISBN,BookTitle,BookAuthor,YearOfPublication,Publisher
0,25030,"""Classical Mythology""",68446,566,10846
1,73,"""Clara Callan""",84385,565,6600
2,8212,"""Decision in Normandy""",17047,555,6603
3,60203,"""Flu: The Story of the Great Influenza Pandemi...",38208,563,5260
4,71716,"""The Mummies of Urumchi""",29206,563,15610


In [5]:
def pick_top_book(user):
    temp_df = book_ratings[book_ratings.userid == user]
    temp_df.sort_values(by='BookRatings', ascending=False)
    temp_df = temp_df.iloc[0]
    temp_dff = books_info[books_info['ISBN'] == temp_df['ISBN']]
    return temp_dff

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np    
    
def contentbasedFiltering(user):
    temp_dff = pick_top_book(user)
    priority = []
    for test_index, item in books_info.iterrows():
        if item.ISBN == temp_dff.ISBN.values:
            continue
        else:
            temp = []
            y = np.array([item['ISBN'], item['BookAuthor'], item['YearOfPublication'], item['Publisher']]).reshape(1,-1)
            x = [temp_dff['ISBN'].values, temp_dff['BookAuthor'].values, temp_dff['YearOfPublication'].values, temp_dff['Publisher'].values]
            x = np.array(x).reshape(1,-1)
            mat = cosine_similarity(x,y)
            sim = mat[0][0]
            temp.append(item.BookTitle)
            temp.append(sim)
            priority.append(temp)
    priority.sort(key=lambda x:x[1])
    priority = priority[::-1]
    print("Following are the recommended books, based on content based filtering approach:")
    for i in range (0,5):
        print(priority[i][0])
    
contentbasedFiltering(73842)

Following are the recommended books, based on content based filtering approach:
"Passionate Possession"
"Yesterday's Echoes"
"Desire Never Changes"
"A Reason For Marriage (Harlequin Collector's Edition)"
"Stranger From The Past (Harlequin Collector's Edition)"
